In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import json
from collections import defaultdict
import nltk
from gensim.models import Word2Vec
import random
import torch.nn.functional as F
import numpy as np

# TODO
Build the RNN 

In [2]:
data = defaultdict(list)
with open("../review.json", "r") as file:
    for line in file:
        d = json.loads(line)
        data[d['funny'] > 0].append((d['review_id'], d['text'], d['funny'] > 0))

In [3]:
num_samples = 60000

In [4]:
# Sample the data
sentences = random.sample(data[True], num_samples)
sentences.extend(random.sample(data[False], num_samples))
random.shuffle(sentences)

del data[True]
del data[False]

In [5]:
# Parse the sentences
def parseSentenceBasic(sentence):
    sent = sentence[1].strip().split()
    return [x.lower() for x in sent]

def parseSentences(sentences, p_func):
    corpus = []
    for s in sentences:
        corpus.append(p_func(s))
    return corpus

In [6]:
# Create the word embedding model based on the current corpus
corpus = parseSentences(sentences, parseSentenceBasic)
word2Vec = Word2Vec(corpus, size=256, min_count=1)

In [7]:
def getWordEmbedding(word, word2Vec):
    try:
        return torch.from_numpy(word2Vec.wv[word])
    except KeyError:
        return torch.from_numpy(word2Vec.wv["<UNK>"])
    
def getWordEmbedding2(word, word2Vec):
    try:
        return word2Vec.wv[word]
    except KeyError:
        return word2Vec.wv["<UNK>"]

# Baseline Network
TODO: Figure out what you actually need to do lol<br>
So far, I have the network outlined, but we need to figure out how to train the data<br>
We also need to figure out how to define a loss function

In [8]:
device = torch.device("cuda:0")

In [ ]:
class ShallowNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(256, 100)
        self.hidden2 = nn.Linear(100, 150)
        self.output = nn.Linear(150, 2)
    
    def forward(self, X):
        X.to(device)
        
        q1 = self.hidden1(X)
        h1 = F.relu(q1)
        q2 = self.hidden2(h1)
        h2 = F.relu(q2)
        
        return self.output(h2)

In [23]:
shallow_net

ShallowNetwork(
  (hidden1): Linear(in_features=256, out_features=100, bias=True)
  (hidden2): Linear(in_features=100, out_features=150, bias=True)
  (output): Linear(in_features=150, out_features=2, bias=True)
)

In [11]:
torch.cuda.is_available()

True

In [12]:
# Turn the corpus into a list of vectors for input
def reviewToVecNaive(s):
    result = torch.zeros(len(s),256)
    for i in range(len(s)):
        result[i] = getWordEmbedding(s[i], word2Vec)
    return torch.sum(result, 0)

def reviewToVecNaive2(s):
    result = torch.zeros(256)
    for i in range(len(s)):
        result = result.add(getWordEmbedding(s[i], word2Vec))
    return result

def reviewToVecNaive3(s):
    result = np.zeros((len(s), 256), dtype=np.float32)
    for i in range(len(s)):
        result[i] = getWordEmbedding(s[i], word2Vec)
    return torch.from_numpy(np.sum(result, axis=0))

In [13]:
N = len(corpus)
corpusVecs1 = [reviewToVecNaive3(s) for s in corpus[:N]]

In [45]:
trainIdx = 20000

In [128]:
from torch import optim
from torch.autograd import Variable

shallow_net = ShallowNetwork()
shallow_net.cuda()
loss_function = nn.CrossEntropyLoss()

def printOptInfo(optimizer):
    print(optimizer.__class__.__name__, optimizer.param_groups[0]['lr'])

def trainShallowNet(optimizer, epochs, corpusVecs, printResults=True):
    if printResults:
        printOptInfo(optimizer)
    for epoch in range(epochs): 
        train_loss = []
        shallow_net.train()
        total_loss = total_correct = 0
        total = trainIdx

        for i in range(trainIdx):
            optimizer.zero_grad()

            corpusVecs[i] = corpusVecs[i].to(device)

            output = shallow_net(corpusVecs[i])
            output = torch.unsqueeze(output, 0)
            output = output.to(device)

            _, x = output.data.topk(1)
            target = torch.tensor([int(sentences[i][2])])
            target = target.to(device)

            if x[0][0] == target:
                total_correct += 1

            loss = loss_function(output, target)
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
            
        if printResults:
            print("epoch: {}, loss: {}, accuracy: {}".format(epoch, np.mean(train_loss), total_correct/total))

In [63]:
def printOptInfo(optimizer):
    print(optimizer.__class__.__name__, optimizer.param_groups[0]['lr'])

In [66]:
# Testing results for SDG optimizer with constant lr, diff epochs
shallow_net = ShallowNetwork().cuda()
o1 = optim.SGD(shallow_net.parameters(), lr=0.001)
trainShallowNet(o1, 2, corpusVecs1)

shallow_net = ShallowNetwork().cuda()
o2 = optim.SGD(shallow_net.parameters(), lr=0.001)
trainShallowNet(o2, 8, corpusVecs1)

SGD 0.001
epoch: 0, loss: 0.6765921560894698, accuracy: 0.5942
epoch: 1, loss: 0.6789048265516758, accuracy: 0.57205
SGD 0.001
epoch: 0, loss: 0.6757887674849481, accuracy: 0.58495
epoch: 1, loss: 0.6723869769826532, accuracy: 0.592
epoch: 2, loss: 0.6770411373376847, accuracy: 0.5788
epoch: 3, loss: 0.677973353369534, accuracy: 0.57975
epoch: 4, loss: 0.6783318033911288, accuracy: 0.5763
epoch: 5, loss: 0.680336127974838, accuracy: 0.57725
epoch: 6, loss: 0.679368626422435, accuracy: 0.58505
epoch: 7, loss: 0.6716068382002414, accuracy: 0.6017


In [76]:
shallow_net.zero_grad()

In [79]:
# Testing results for SDG optimizer with constant epochs, diff lr
shallow_net = ShallowNetwork().cuda()
o3 = optim.SGD(shallow_net.parameters(), lr=0.0001)
trainShallowNet(o3, 2, corpusVecs1)

shallow_net = ShallowNetwork().cuda()
o4 = optim.SGD(shallow_net.parameters(), lr=0.01)
trainShallowNet(o4, 2, corpusVecs1)

shallow_net = ShallowNetwork().cuda()
o5 = optim.SGD(shallow_net.parameters(), lr=0.1)
trainShallowNet(o5, 2, corpusVecs1)

SGD 0.0001
epoch: 0, loss: 0.697416302045621, accuracy: 0.60095
epoch: 1, loss: 0.6538178123567253, accuracy: 0.62615
SGD 0.01
epoch: 0, loss: nan, accuracy: 0.0195
epoch: 1, loss: nan, accuracy: 0.0
SGD 0.1
epoch: 0, loss: nan, accuracy: 0.00335
epoch: 1, loss: nan, accuracy: 0.0


In [81]:
shallow_net = ShallowNetwork().cuda()
o_best_sgd = optim.SGD(shallow_net.parameters(), lr=0.0001)
trainShallowNet(o_best_sgd, 10, corpusVecs1)

SGD 0.0001
epoch: 0, loss: 0.7003838335053064, accuracy: 0.5994
epoch: 1, loss: 0.6560078274384141, accuracy: 0.62375
epoch: 2, loss: 0.6496948119603098, accuracy: 0.6324
epoch: 3, loss: 0.6455893912747502, accuracy: 0.6383
epoch: 4, loss: 0.6431354025945067, accuracy: 0.6404
epoch: 5, loss: 0.6400601583058014, accuracy: 0.64185
epoch: 6, loss: 0.6372200701104477, accuracy: 0.64595
epoch: 7, loss: 0.6354311815664172, accuracy: 0.6462
epoch: 8, loss: 0.633373790150322, accuracy: 0.64905
epoch: 9, loss: 0.6313983280704357, accuracy: 0.6485


In [80]:
# Testing results with Adam with 2 epochs
shallow_net = ShallowNetwork().cuda()
o6 = optim.Adam(shallow_net.parameters(), lr=0.1, betas=(0.9, 0.99)) # Starting with the best results found in the paper
trainShallowNet(o6, 2, corpusVecs1)

shallow_net = ShallowNetwork().cuda()
o7 = optim.Adam(shallow_net.parameters(), lr=0.01) # Starting with the best results found in the paper
trainShallowNet(o7, 2, corpusVecs1)

Adam 0.1
epoch: 0, loss: 606.7561558526859, accuracy: 0.49975
epoch: 1, loss: 4.72142732155174, accuracy: 0.4976
Adam 0.01
epoch: 0, loss: 3.656859737342596, accuracy: 0.499
epoch: 1, loss: 0.7507467496782542, accuracy: 0.4952


In [82]:
# Testing results with Adam with more epochs using best parameters from before
shallow_net = ShallowNetwork().cuda()
o_best_adam = optim.Adam(shallow_net.parameters(), lr=0.1, betas=(0.9, 0.99)) # Starting with the best results found in the paper
trainShallowNet(o_best_adam, 8, corpusVecs1)

Adam 0.1
epoch: 0, loss: 325.3814041033566, accuracy: 0.4988
epoch: 1, loss: 0.7207487521916628, accuracy: 0.49715
epoch: 2, loss: 0.7207487520843744, accuracy: 0.49715
epoch: 3, loss: 0.7207487520709633, accuracy: 0.49715
epoch: 4, loss: 0.7207487522661686, accuracy: 0.49715
epoch: 5, loss: 0.7207487523317337, accuracy: 0.49715
epoch: 6, loss: 0.7207487522795797, accuracy: 0.49715
epoch: 7, loss: 0.7207487520948053, accuracy: 0.49715


## After playing around with the training set, apply results to large test set

In [155]:
# Test accuracy
testIdx = len(corpusVecs1)

def testShallowNet(corpusVecs, printResults=True):
    shallow_net.eval()
    if printResults:
        print("Testing Validation set")
        
    valid_loss = []
    vtc = 0
    total = testIdx - trainIdx

    for i in range(trainIdx, testIdx):
        corpusVecs[i] = corpusVecs[i].to(device)

        output = shallow_net(corpusVecs1[i])
        output = torch.unsqueeze(output,0)
        output = output.to(device)

        _, x = output.data.topk(1)
        target = torch.tensor([int(sentences[i][2])])
        target = target.to(device)

        if x[0][0] == target:
            vtc += 1

        loss = loss_function(output, target)
        valid_loss.append(loss.item())

    if printResults:
        print ("Valid Loss: ", np.mean(valid_loss), " Valid Accuracy: ", vtc/total)

# TODO: Run this overnight b/c it's a lot of epochs lmao

In [160]:
shallow_net = ShallowNetwork().cuda()
o_best_sgd = optim.SGD(shallow_net.parameters(), lr=0.0001)
trainShallowNet(o_best_sgd, 50, corpusVecs1)
testShallowNet(corpusVecs1)

SGD 0.0001
epoch: 0, loss: 0.7029884145474061, accuracy: 0.59845
epoch: 1, loss: 0.6546152041204274, accuracy: 0.6256
epoch: 2, loss: 0.6473501835002564, accuracy: 0.63295
epoch: 3, loss: 0.6460502143951133, accuracy: 0.63695
epoch: 4, loss: 0.6417039959196933, accuracy: 0.64185
epoch: 5, loss: 0.6399227172822691, accuracy: 0.64245
epoch: 6, loss: 0.637738271092996, accuracy: 0.6452
epoch: 7, loss: 0.6356239701982588, accuracy: 0.648
epoch: 8, loss: 0.6344645050564781, accuracy: 0.6472
epoch: 9, loss: 0.6316201680244878, accuracy: 0.65185
epoch: 10, loss: 0.631434486806579, accuracy: 0.64915
epoch: 11, loss: 0.6294749275523238, accuracy: 0.6513
epoch: 12, loss: 0.6292330946765841, accuracy: 0.65115
epoch: 13, loss: 0.6256487735946663, accuracy: 0.6526
epoch: 14, loss: 0.6239506203782744, accuracy: 0.6581
epoch: 15, loss: 0.6237775511654093, accuracy: 0.6561
epoch: 16, loss: 0.6213585338531993, accuracy: 0.6591
epoch: 17, loss: 0.6192147739214823, accuracy: 0.6623
epoch: 18, loss: 0.617

# Other baseline models

## Remove Punctuation

In [98]:
### Remove Punctuation ###
import string
def parseSentenceRemovePunc(sentence):
    sent = sentence[1]
    sent = sent.translate(str.maketrans('', '', string.punctuation))
    sent = sent.strip().split()
    return [x.lower() for x in sent]

def reviewToVecRemovePunc(s):
    result = np.zeros((len(s), 256), dtype=np.float32)
    for i in range(len(s)):
        result[i] = getWordEmbedding(s[i], word2VecRP)
    return torch.from_numpy(np.sum(result, axis=0))

In [102]:
parseSentenceRemovePunc((1,"I'm so pumped!!! for the runn* lol yeet., hi, there"))

['im', 'so', 'pumped', 'for', 'the', 'runn', 'lol', 'yeet', 'hi', 'there']

In [104]:
corpusRP = parseSentences(sentences, parseSentenceRemovePunc)
word2VecRP = Word2Vec(corpusRP, size=256, min_count=1)
corpusVecsRP = [reviewToVecRemovePunc(s) for s in corpusRP]

In [129]:
# Initial exploration of Removing Punctuation
shallow_net = ShallowNetwork().cuda()
o = optim.SGD(shallow_net.parameters(), lr=0.0001)
trainShallowNet(o, 10, corpusVecsRP)

SGD 0.0001
epoch: 0, loss: 0.6904180389977992, accuracy: 0.60665
epoch: 1, loss: 0.6519244625438004, accuracy: 0.6303
epoch: 2, loss: 0.6461100714338943, accuracy: 0.63745
epoch: 3, loss: 0.6429648605552735, accuracy: 0.6404
epoch: 4, loss: 0.6395431984181515, accuracy: 0.6441
epoch: 5, loss: 0.6372030795700848, accuracy: 0.64735
epoch: 6, loss: 0.6342780151852989, accuracy: 0.64745
epoch: 7, loss: 0.6318999478915707, accuracy: 0.6513
epoch: 8, loss: 0.6307564444663003, accuracy: 0.6503
epoch: 9, loss: 0.6280690464532003, accuracy: 0.65265


# TODO Run overnight

In [157]:
# Testing model wtih 50 epochs
o = optim.SGD(shallow_net.parameters(), lr=0.0001)
trainShallowNet(o, 50, corpusVecsRP)
testShallowNet(corpusVecsRP)

epoch: 0, loss: 0.6832736623609439, accuracy: 0.61335
epoch: 1, loss: 0.6423848252095282, accuracy: 0.63495
epoch: 2, loss: 0.6355594107333571, accuracy: 0.6428
epoch: 3, loss: 0.6304630756832659, accuracy: 0.6475
epoch: 4, loss: 0.6258434915305116, accuracy: 0.6512
epoch: 5, loss: 0.6209469932716339, accuracy: 0.6549
epoch: 6, loss: 0.6169257361199707, accuracy: 0.65915
epoch: 7, loss: 0.6140793929651379, accuracy: 0.65965
epoch: 8, loss: 0.6096197547988966, accuracy: 0.6651
epoch: 9, loss: 0.6069094866424799, accuracy: 0.66645
epoch: 10, loss: 0.602978558184579, accuracy: 0.6693
epoch: 11, loss: 0.5994665628220885, accuracy: 0.67195
epoch: 12, loss: 0.5957826994009316, accuracy: 0.67455
epoch: 13, loss: 0.5939640895336866, accuracy: 0.67545
epoch: 14, loss: 0.5888997744682245, accuracy: 0.68005
epoch: 15, loss: 0.5852831742266194, accuracy: 0.68205
epoch: 16, loss: 0.5824117169490084, accuracy: 0.6848
epoch: 17, loss: 0.5789790781266987, accuracy: 0.68625
epoch: 18, loss: 0.575621324

## Punctuation Embedding

In [122]:
### Make punctuation an embedding ###
def parseSentencePuncEmbed(sentence):
    sent = sentence[1]
    punc = ['.', '!', ';', '?', '-']
    for p in punc:  
        sent = sent.replace(p, " " + p + " ")
    sent = sent.replace("'", "")
    sent = sent.strip().split()
    return [x.lower() for x in sent]

def reviewToVecPuncEmbed(s):
    result = np.zeros((len(s), 256), dtype=np.float32)
    for i in range(len(s)):
        result[i] = getWordEmbedding(s[i], word2VecPE)
    return torch.from_numpy(np.sum(result, axis=0))

In [126]:
corpusPE = parseSentences(sentences, parseSentencePuncEmbed)
word2VecPE = Word2Vec(corpusPE, size=256, min_count=1)
corpusVecsPE = [reviewToVecPuncEmbed(s) for s in corpusPE]

In [131]:
# TODO Perform the training
shallow_net = ShallowNetwork().cuda()
o = optim.SGD(shallow_net.parameters(), lr=0.0001)
trainShallowNet(o, 10, corpusVecsPE)

SGD 0.0001
epoch: 0, loss: 0.69196751480503, accuracy: 0.6056
epoch: 1, loss: 0.6530303657909855, accuracy: 0.6277
epoch: 2, loss: 0.648223162284866, accuracy: 0.631
epoch: 3, loss: 0.6443316187039018, accuracy: 0.63725
epoch: 4, loss: 0.6412390139104799, accuracy: 0.6419
epoch: 5, loss: 0.6398565146304667, accuracy: 0.64215
epoch: 6, loss: 0.6372144710576162, accuracy: 0.64715
epoch: 7, loss: 0.6358025217071176, accuracy: 0.64815
epoch: 8, loss: 0.6339809098771307, accuracy: 0.64985
epoch: 9, loss: 0.6329009722508258, accuracy: 0.6497


# TODO Run overnight

In [158]:
shallow_net = ShallowNetwork().cuda()
o = optim.SGD(shallow_net.parameters(), lr=0.0001)
trainShallowNet(o, 50, corpusVecsPE)
testShallowNet(corpusVecsPE)

SGD 0.0001
epoch: 0, loss: 0.6889368819605326, accuracy: 0.6038
epoch: 1, loss: 0.652522301142104, accuracy: 0.6279
epoch: 2, loss: 0.6455427042042836, accuracy: 0.6367
epoch: 3, loss: 0.6412872657289729, accuracy: 0.64255
epoch: 4, loss: 0.6383467690318823, accuracy: 0.6432
epoch: 5, loss: 0.6365085242732428, accuracy: 0.6458
epoch: 6, loss: 0.6344501096766442, accuracy: 0.64835
epoch: 7, loss: 0.6320376964886673, accuracy: 0.6479
epoch: 8, loss: 0.629931101786159, accuracy: 0.6513
epoch: 9, loss: 0.6289591054193675, accuracy: 0.6521
epoch: 10, loss: 0.6264607416269835, accuracy: 0.65455
epoch: 11, loss: 0.6252018424151465, accuracy: 0.65695
epoch: 12, loss: 0.6241470107048052, accuracy: 0.65575
epoch: 13, loss: 0.6229727788352407, accuracy: 0.6592
epoch: 14, loss: 0.6204118521956727, accuracy: 0.6614
epoch: 15, loss: 0.619339306929987, accuracy: 0.6624
epoch: 16, loss: 0.6165382479340303, accuracy: 0.66215
epoch: 17, loss: 0.615444421865046, accuracy: 0.6645
epoch: 18, loss: 0.613253

## TF-IDF Model

In [134]:
# Making tf-idf corpus dict
from collections import defaultdict
tfidf_dict = defaultdict(int)

for i in range(len(sentences)):
    for word in parseSentencePuncEmbed(sentences[i]):
        tfidf_dict[word] += 1

In [151]:
### TF-IDF Parsing ###
import math
N = len(sentences)
def reviewToVecTFIDF(s):
    result = np.zeros((len(s), 256), dtype=np.float32)

    doc_counts = defaultdict(int)
    for word in s:
        doc_counts[word] += 1
    
    for i in range(len(s)):
        weight = torch.tensor(doc_counts[s[i]] * math.log(N/tfidf_dict[s[i]]))
        word_embedding = getWordEmbedding(s[i], word2VecPE)
        result[i] = word_embedding*weight.expand_as(word_embedding)
        
    return torch.from_numpy(np.sum(result, axis=0))

In [152]:
corpusVecsTFIDF = [reviewToVecTFIDF(s) for s in corpusPE]

In [153]:
# TODO Perform the training
shallow_net = ShallowNetwork().cuda()
o = optim.SGD(shallow_net.parameters(), lr=0.0001)
trainShallowNet(o, 10, corpusVecsTFIDF)

SGD 0.0001
epoch: 0, loss: 1.3747586511999368, accuracy: 0.54925
epoch: 1, loss: 1.2956613364724443, accuracy: 0.53415
epoch: 2, loss: 21.70679119032817, accuracy: 0.5371
epoch: 3, loss: 5992.674446704036, accuracy: 0.5161
epoch: 4, loss: 0.9042090472906827, accuracy: 0.49825
epoch: 5, loss: 0.7160526506245136, accuracy: 0.49975
epoch: 6, loss: 0.6929773669540882, accuracy: 0.5011
epoch: 7, loss: 0.6929947181999684, accuracy: 0.5026
epoch: 8, loss: 0.6929855198308825, accuracy: 0.50265
epoch: 9, loss: 0.6929498706966638, accuracy: 0.50265


# TODO Run Overnight

In [159]:
shallow_net = ShallowNetwork().cuda()
o = optim.SGD(shallow_net.parameters(), lr=0.0001)
trainShallowNet(o, 50, corpusVecsTFIDF)
testShallowNet(corpusVecsTFIDF)

SGD 0.0001
epoch: 0, loss: 1.5166704046139494, accuracy: 0.5327
epoch: 1, loss: 1.5156483787558972, accuracy: 0.5381
epoch: 2, loss: 1.3860277008742095, accuracy: 0.54615
epoch: 3, loss: 0.6912884895667434, accuracy: 0.53885
epoch: 4, loss: 0.6869653450384736, accuracy: 0.5336
epoch: 5, loss: 0.6829224729686976, accuracy: 0.5572
epoch: 6, loss: 0.6818263136737048, accuracy: 0.569
epoch: 7, loss: 0.6803631385959685, accuracy: 0.56845
epoch: 8, loss: 0.6787210705757141, accuracy: 0.5814
epoch: 9, loss: 0.6782761797986925, accuracy: 0.58095
epoch: 10, loss: 0.6777408777654171, accuracy: 0.57755
epoch: 11, loss: 0.6774849384561181, accuracy: 0.5803
epoch: 12, loss: 0.6763261337310076, accuracy: 0.58055
epoch: 17, loss: 0.6744417943183333, accuracy: 0.5817
epoch: 18, loss: 0.6731176926750689, accuracy: 0.5843
epoch: 19, loss: 0.6730973766155541, accuracy: 0.58605
epoch: 20, loss: 0.6727174837231636, accuracy: 0.58805
epoch: 21, loss: 0.6717091576907783, accuracy: 0.58765
epoch: 22, loss: 0.